In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import folium
import os
import glob
from pandas.errors import EmptyDataError
import matplotlib.pyplot as plt
from functions import smooth
import plotly.subplots as sp
import plotly.graph_objs as go
import seaborn as sns
import plotly.io as pio
from scipy.signal import find_peaks

In [ ]:
# Path
filepath_sensors = './Data/input/'
output_filepath = './Data/output/'
path_img = './images/'
path_viz = './Visualization/'

if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [ ]:

overview = pd.read_csv('./Data/overview.csv')
fig, ax = plt.subplots(figsize = (12,5))
# Load the example tips dataset
# tips = sns.load_dataset("tips")
#
# Draw a nested boxplot to show bills by day and time
sns.boxplot(x="scenario", y="age", hue="sex", palette=["r", "b"],data=overview[(overview.sex=="female" ) | (overview.sex=="male")])

sns.set_theme(style="whitegrid")

sns.despine(offset=10, trim=True)
# plt.savefig(path_img + 'Manipulation vs sex(BoxPlot).pdf',bbox_inches='tight')

In [ ]:
# path_E4 = "./data/output/"
participant_filepath_output = "./data/output/"
for participant_filepath in glob.glob(participant_filepath_output + 'amer4/', recursive=True):
    accelerometer = pd.read_csv('{}Accelerometer_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    # df = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')
    accelerometer.index = pd.to_datetime(accelerometer['time'], unit = 'ns')

    fig = go.Figure()

    for axis in ['x', 'y', 'z']:
        fig.add_trace(go.Scatter(x = accelerometer.index, y = accelerometer[axis], name = axis))
    fig.update_layout(legend_title_text = "ACC (m/s^2)", autosize=True)
    fig.update_xaxes(title_text="Timestamp_" + participant)
    fig.update_yaxes(title_text="ACC")
        # matplotlib.rcParams['figure.figsize'] = [40, 10]
    fig.show()

In [ ]:
path_viz = './Visualization/Specific_activity/'

In [ ]:
# Define the category to visualize
# category = 'walking'
# Define the start and end indices for the data range to plot
start_index = 6300
end_index = 7000
i = -1
j = -1
# Define a list of sensor data files to process
sensor_files = ['Accelerometer_annotated.csv', 'Gyroscope_annotated.csv', 'Magnetometer_annotated.csv']
# Define a list of categories to visualize
categories = ['walking', 'standing']
# , 'PtTX', 'WWTX', 'SWTX', 'TXtP', 'TXtTL', 'WWTL', 'SWTL'
# Loop through all sensor data files
for sensor_file in sensor_files:
    # Loop through all participant files for the current sensor data file
    for participant_filepath in glob.glob(participant_filepath_output + 'amer2/', recursive=True):
        i+=i+1
        # read in the accelerometer data
        # accelerometer = pd.read_csv('{}Accelerometer_annotated.csv'.format(participant_filepath))
        # read in the sensor data
        sensor_data = pd.read_csv('{}{}'.format(participant_filepath, sensor_file))

        # extract the participant ID from the file path
        components = os.path.dirname(participant_filepath)
        components = components.split(os.path.sep)
        participant = components[-1]
        # Loop through all categories to plot
        for category in categories:
            j = j+1
            # filter the sensor data to include only the rows that correspond to the category
            category_data = sensor_data[sensor_data['text'] == category]
            # plot the specified range of x-, y-, and z-axis data for the sensor
            fig, ax = plt.subplots(figsize=(10,5))
            ax.plot(category_data['x'][start_index:end_index], label='x-axis')
            ax.plot(category_data['y'][start_index:end_index], label='y-axis')
            ax.plot(category_data['z'][start_index:end_index], label='z-axis')
            ax.set_title('{} Data for {}'.format(sensor_file.replace('_annotated.csv', ''), category))
            ax.set_xlabel('Sample')
            ax.set_ylabel('Value')
            ax.legend()
            # plt.savefig(path_viz + sensor_files[i] + category[j] + '.pdf',bbox_inches='tight')
            plt.savefig(path_viz + '{}_{}.pdf'.format(i, j), bbox_inches='tight')
            # ax.set_xlim(0, end_index - start_index)
            plt.show()

In [ ]:
import pandas as pd

# Load original dataframe
df = pd.read_csv('./Data/input/amer/Accelerometer.csv')
# overview = pd.read_csv('./Data/overview.csv')
# Convert timestamp column to datetime format
df['timestamp'] = pd.to_datetime(df['time'], unit='ns')

# Format datetime column to string format
df['timestamp'] = df['timestamp'].dt.strftime('%m/%d/%Y %I:%M:%S %p')

# Create new dataframe with formatted timestamp column
new_df = pd.DataFrame({'Timestamp': df['timestamp'],'ACC-X': df['x'], 'ACC-Y': df['y'],'ACC-Z': df['z']})

# Save new dataframe to CSV file
new_df.to_csv('./Data/input/amer/formatted_data.csv', index=False)

In [ ]:
# Load original dataframe
df = pd.read_csv('./Data/input/amer/Annotation.csv')
# overview = pd.read_csv('./Data/overview.csv')
# Convert timestamp column to datetime format
df['timestamp'] = pd.to_datetime(df['time'], unit='ns')

# Format datetime column to string format
df['timestamp'] = df['timestamp'].dt.strftime('%m/%d/%Y %I:%M:%S %p')

# Create new dataframe with formatted timestamp column
new_df = pd.DataFrame({'Timestamp': df['timestamp'],'Label': df['text']})

# Save new dataframe to CSV file
new_df.to_csv('./Data/input/amer/labelllled_data.csv', index=False)

In [ ]:
# path_E4 = "./data/output/"
participant_filepath_output = "./old-data/OUTPUT/"
for participant_filepath in glob.glob(participant_filepath_output + 'amer3/', recursive=True):
    accelerometer = pd.read_csv('{}Accelerometer_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    # df = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')
    accelerometer.index = pd.to_datetime(accelerometer['time'], unit = 'ns')
    accelerometer = accelerometer.dropna(subset=['text'])

    # Create a figure with 3 subplots using the subplots function
    fig = sp.make_subplots(rows=3, cols=1)

    # Add a trace for each axis, using the Scatter function
    fig.add_trace(go.Scatter(x=accelerometer.index, y=accelerometer["x"], text = accelerometer["text"], name='X-axis'), row=1, col=1)
    fig.add_trace(go.Scatter(x=accelerometer.index, y=accelerometer["y"], text = accelerometer["text"], name='Y-axis'), row=2, col=1)
    fig.add_trace(go.Scatter(x=accelerometer.index, y=accelerometer["z"], text = accelerometer["text"], name='Z-axis'), row=3, col=1)
    # Initialize a list to hold the annotations
    annotations = []

    # Iterate through the data and add an annotation when the label changes
    for i in range(3):
        prev_label = ''
        for j, point in enumerate(fig.data[0].text):
            if point != prev_label:
                annotations.append({
                    'x': fig.data[0].x[j],
                    'y': fig.data[0].y[j],
                    'text': point,
                    'xref': 'x',
                    'yref': 'y'
                })
            prev_label = point

        # Add the annotations to the layout
        # fig.update_layout(annotations=annotations)

    # Customize the layout
    fig.update_layout(title='3-Axis Accelerometer Data (m/s^2)')

    # Set the height and width of the subplots
    # fig.update_layout(autosize=True)

    # Set the name of each axis
    fig.update_layout(yaxis1=dict(title='x'),
                      yaxis2=dict(title='y'),
                      yaxis3=dict(title='z'),
                      xaxis1=dict(title=''),
                      xaxis2=dict(title=''),
                      xaxis3=dict(title='Time'))
    # Show the figure
    fig.show()
    pio.write_image(fig, path_viz+'ACC.pdf')
    # for i, trace in enumerate(fig.data):
    #     color = trace.marker.color
    #     subfig = go.Figure(trace)
    #     subfig.update_traces(marker=dict(color=color))
    #     pio.write_image(subfig, path_viz + f'figure{i}.png')

In [ ]:
# path_E4 = "./data/output/"
participant_filepath_output = "./Data/output/"
for participant_filepath in glob.glob(participant_filepath_output + '2/', recursive=True):
    accelerometer = pd.read_csv('{}Orientation_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    # df = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')
    accelerometer.index = pd.to_datetime(accelerometer['Timestamp'], unit = 'ns')
    accelerometer = accelerometer.dropna(subset=['Label'])
    #
    # # Create a figure with 3 subplots using the subplots function
    fig = sp.make_subplots(rows=3, cols=1)

    # Add a trace for each axis, using the Scatter function
    fig.add_trace(go.Scatter(x=accelerometer.index, y=accelerometer["roll"], text = accelerometer["Label"], name='X-axis'), row=1, col=1)
    fig.add_trace(go.Scatter(x=accelerometer.index, y=accelerometer["pitch"], text = accelerometer["Label"], name='Y-axis'), row=2, col=1)
    fig.add_trace(go.Scatter(x=accelerometer.index, y=accelerometer["yaw"], text = accelerometer["Label"], name='Z-axis'), row=3, col=1)
    # Initialize a list to hold the annotations
    annotations = []

    # Iterate through the data and add an annotation when the label changes
    # for i in range(3):
    #     prev_label = ''
    #     for j, point in enumerate(fig.data[0].text):
    #         if point != prev_label:
    #             annotations.append({
    #                 'x': fig.data[0].x[j],
    #                 'y': fig.data[0].y[j],
    #                 'text': point,
    #                 'xref': 'x',
    #                 'yref': 'y'
    #             })
    #         prev_label = point

        # Add the annotations to the layout
        # fig.update_layout(annotations=annotations)

    # Customize the layout
    fig.update_layout(title='3-Axis Orientation Data (rad)')

    # Set the height and width of the subplots
    # fig.update_layout(autosize=True)

    # Set the name of each axis
    fig.update_layout(yaxis1=dict(title='roll'),
                      yaxis2=dict(title='pitch'),
                      yaxis3=dict(title='yaw'),
                      xaxis1=dict(title=''),
                      xaxis2=dict(title=''),
                      xaxis3=dict(title='Timestamp'))
    # Show the figure
    fig.show()
    pio.write_image(fig, path_viz+'ORI.pdf')

In [ ]:
participant_filepath_output = "./data/output/"
for participant_filepath in glob.glob(participant_filepath_output + '*/', recursive=True):
    accelerometer = pd.read_csv('{}Accelerometer_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    # df = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')
    accelerometer.index = pd.to_datetime(accelerometer['Timestamp'], unit = 'ns')
    accelerometer = accelerometer.dropna(subset=['Label'])
    # Select only the 'x', 'y', and 'z' columns
    data = accelerometer[['x', 'y', 'z']]

    # Group the data by 'Category'
    grouped_data = data.groupby(accelerometer['Label'])

    # Calculate the statistics for each group
    stats = grouped_data.apply(lambda x: pd.Series({"x_min":x.x.min(),"y_min":x.y.min(),"z_min":x.z.min(),"x_max":x.x.max(),"y_max":x.y.max(),"z_max":x.z.max(),"x_mean":x.x.mean(),"y_mean":x.y.mean(),"z_mean":x.z.mean(),"x_std":x.x.std(),"y_std":x.y.std(),"z_std":x.z.std()}))

    display(stats)

In [ ]:
participant_filepath_output = "./data/output/"
all_stats = pd.DataFrame()
for participant_filepath in glob.glob(participant_filepath_output + '*/', recursive=True):
    accelerometer = pd.read_csv('{}Accelerometer_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    accelerometer.index = pd.to_datetime(accelerometer['Timestamp'], unit = 'ns')
    accelerometer = accelerometer.dropna(subset=['Label'])
    # Select only the 'x', 'y', and 'z' columns
    data = accelerometer[['x', 'y', 'z']]
    # Group the data by 'Category'
    grouped_data = data.groupby(accelerometer['Label'])
    # Calculate the statistics for each group
    stats = grouped_data.agg(["min", "max", "mean", "std"])
    stats['participant'] = participant
    all_stats = all_stats.append(stats)
max_stats = all_stats.groupby(level=0).max().round(2)
display(max_stats)
max_stats.to_csv ('./data/max_ACC.csv')

In [ ]:
participant_filepath_output = "./data/output/"
all_stats = pd.DataFrame()
for participant_filepath in glob.glob(participant_filepath_output + '*/', recursive=True):
    accelerometer = pd.read_csv('{}Orientation_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    accelerometer.index = pd.to_datetime(accelerometer['Timestamp'], unit = 'ns')
    accelerometer = accelerometer.dropna(subset=['Label'])
    # Select only the 'x', 'y', and 'z' columns
    data = accelerometer[['roll', 'pitch', 'yaw']]
    # Group the data by 'Category'
    grouped_data = data.groupby(accelerometer['Label'])
    # Calculate the statistics for each group
    stats = grouped_data.agg(["min", "max", "mean", "std"])
    stats['participant'] = participant
    all_stats = all_stats.append(stats)
max_stats = all_stats.groupby(level=0).max().round(2)
display(max_stats)
max_stats.to_csv ('./data/max_Orient.csv')

In [ ]:
participant_filepath_output = "./data/output/"
all_stats = pd.DataFrame()
for participant_filepath in glob.glob(participant_filepath_output + '*/', recursive=True):
    accelerometer = pd.read_csv('{}Gyroscope_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    accelerometer.index = pd.to_datetime(accelerometer['Timestamp'], unit = 'ns')
    accelerometer = accelerometer.dropna(subset=['Label'])
    # Select only the 'x', 'y', and 'z' columns
    data = accelerometer[['x', 'y', 'z']]
    # Group the data by 'Category'
    grouped_data = data.groupby(accelerometer['Label'])
    # Calculate the statistics for each group
    stats = grouped_data.agg(["min", "max", "mean", "std"])
    stats['participant'] = participant
    all_stats = all_stats.append(stats)
max_stats = all_stats.groupby(level=0).max().round(2)
display(max_stats)
max_stats.to_csv ('./data/max_Gyro.csv')

In [ ]:
participant_filepath_output = "./data/output/"
all_stats = pd.DataFrame()
for participant_filepath in glob.glob(participant_filepath_output + '*/', recursive=True):
    accelerometer = pd.read_csv('{}Magnetometer_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    accelerometer.index = pd.to_datetime(accelerometer['Timestamp'], unit = 'ns')
    accelerometer = accelerometer.dropna(subset=['Label'])
    # Select only the 'x', 'y', and 'z' columns
    data = accelerometer[['x', 'y', 'z']]
    # Group the data by 'Category'
    grouped_data = data.groupby(accelerometer['Label'])
    # Calculate the statistics for each group
    stats = grouped_data.agg(["min", "max", "mean", "std"])
    stats['participant'] = participant
    all_stats = all_stats.append(stats)
max_stats = all_stats.groupby(level=0).max().round(2)
display(max_stats)
max_stats.to_csv ('./data/max_Magno.csv')

In [ ]:
participant_filepath_output = "./data/output/"
all_stats = pd.DataFrame()
for participant_filepath in glob.glob(participant_filepath_output + '*/', recursive=True):
    accelerometer = pd.read_csv('{}Light_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    accelerometer.index = pd.to_datetime(accelerometer['Timestamp'], unit = 'ns')
    accelerometer = accelerometer.dropna(subset=['Label'])
    # Select only the 'x', 'y', and 'z' columns
    data = accelerometer['lux']
    # Group the data by 'Category'
    grouped_data = data.groupby(accelerometer['Label'])
    # Calculate the statistics for each group
    stats = grouped_data.agg(["min", "max", "mean", "std"])
    stats['participant'] = participant
    all_stats = all_stats.append(stats)
max_stats = all_stats.groupby(level=0).max().round(2)
display(max_stats)
max_stats.to_csv ('./data/max_Light.csv')

In [ ]:
# path_E4 = "./data/output/"
participant_filepath_output = "./data/output/"
for participant_filepath in glob.glob(participant_filepath_output + 'amer3/', recursive=True):
    gyroscope = pd.read_csv('{}Light_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    # df = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')
    gyroscope.index = pd.to_datetime(gyroscope['Timestamp'], unit = 'ns')
    gyroscope = gyroscope.dropna(subset=['text'])

    # Create a figure with 3 subplots using the subplots function
    fig = sp.make_subplots(rows=3, cols=1)

    # Add a trace for each axis, using the Scatter function
    fig.add_trace(go.Scatter(x=gyroscope.index, y=gyroscope["x"], text = gyroscope["text"], name='X-axis'), row=1, col=1)
    fig.add_trace(go.Scatter(x=gyroscope.index, y=gyroscope["y"], text = gyroscope["text"], name='Y-axis'), row=2, col=1)
    fig.add_trace(go.Scatter(x=gyroscope.index, y=gyroscope["z"], text = gyroscope["text"], name='Z-axis'), row=3, col=1)
    # Initialize a list to hold the annotations
    annotations = []

    # Iterate through the data and add an annotation when the label changes
    prev_label = ''
    for i, point in enumerate(fig.data[0].text):
        if point != prev_label:
            annotations.append({
                'x': fig.data[0].x[i],
                'y': fig.data[0].y[i],
                'text': point,
                'xref': 'x',
                'yref': 'y'
            })
        prev_label = point

    # Add the annotations to the layout
    # fig.update_layout(annotations=annotations)

    # Customize the layout
    fig.update_layout(title='3-Axis Gyroscope Data (rad/s)')

    # Set the height and width of the subplots
    fig.update_layout(height=800, width=1600)

    # Set the name of each axis
    fig.update_layout(yaxis1=dict(title='x'),
                      yaxis2=dict(title='y'),
                      yaxis3=dict(title='z'),
                      xaxis1=dict(title=''),
                      xaxis2=dict(title=''),
                      xaxis3=dict(title='Time'))
    # Show the figure
    fig.show()
    pio.write_image(fig, path_viz+'GYRO.pdf')
    # for i, trace in enumerate(fig.data):
    #     color = trace.marker.color
    #     subfig = go.Figure(trace)
    #     subfig.update_traces(marker=dict(color=color))
    #     pio.write_image(subfig, path_viz + f'figure{i}.png')

In [ ]:
# path_E4 = "./data/output/"
participant_filepath_output = "./data/output/"
for participant_filepath in glob.glob(participant_filepath_output + 'amer3/', recursive=True):
    magnetometer = pd.read_csv('{}Magnetometer_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    # df = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')
    magnetometer.index = pd.to_datetime(magnetometer['time'], unit = 'ns')
    magnetometer = magnetometer.dropna(subset=['text'])

    # Create a figure with 3 subplots using the subplots function
    fig = sp.make_subplots(rows=3, cols=1)

    # Add a trace for each axis, using the Scatter function
    fig.add_trace(go.Scatter(x=magnetometer.index, y=magnetometer["x"], text = magnetometer["text"], name='X-axis'), row=1, col=1)
    fig.add_trace(go.Scatter(x=magnetometer.index, y=magnetometer["y"], text = magnetometer["text"], name='Y-axis'), row=2, col=1)
    fig.add_trace(go.Scatter(x=magnetometer.index, y=magnetometer["z"], text = magnetometer["text"], name='Z-axis'), row=3, col=1)
    # Initialize a list to hold the annotations
    annotations = []

    # Iterate through the data and add an annotation when the label changes
    prev_label = ''
    for i, point in enumerate(fig.data[0].text):
        if point != prev_label:
            annotations.append({
                'x': fig.data[0].x[i],
                'y': fig.data[0].y[i],
                'text': point,
                'xref': 'x',
                'yref': 'y'
            })
        prev_label = point

    # Add the annotations to the layout
    # fig.update_layout(annotations=annotations)

    # Customize the layout
    fig.update_layout(title='3-Axis Magnetometer Data (μT)')

    # Set the height and width of the subplots
    fig.update_layout(height=800, width=1200)

    # Set the name of each axis
    fig.update_layout(yaxis1=dict(title='x'),
                      yaxis2=dict(title='y'),
                      yaxis3=dict(title='z'),
                      xaxis1=dict(title=''),
                      xaxis2=dict(title=''),
                      xaxis3=dict(title='Time'))
    # Show the figure
    fig.show()
    pio.write_image(fig, path_viz+'MAGNO.pdf')
    # for i, trace in enumerate(fig.data):
    #     color = trace.marker.color
    #     subfig = go.Figure(trace)
    #     subfig.update_traces(marker=dict(color=color))
    #     pio.write_image(subfig, path_viz + f'figure{i}.png')

In [ ]:
df = pd.read_csv('./Data/input/amer4/Accelerometer.csv')
ts = pd.to_datetime(df['time'], unit = 'ns')
period = np.nanmedian(ts.diff().dt.total_seconds())

sp = np.fft.fft(df['x'])
freq = np.fft.fftfreq(len(df.index), period)

mask = (freq >= 0) &(freq < 1)

fig = go.Figure()
fig.add_trace(go.Scatter(x = freq[mask], y = [np.linalg.norm(s) for s in sp[mask]]))
fig.show()

In [ ]:
df = pd.read_csv('./Data/input/amer4/Accelerometer.csv')
aaa = df.drop(['seconds_elapsed'], axis=1)
# aaa = df.set_index("time")
aaa.index = pd.to_datetime(aaa['time'], unit = 'ns')
#
aaa

In [ ]:
# Calculate the derivative of the value column for a time interval of 1 second
df_derivative = aaa.diff(periods=1)
# display(df_derivative)
# df_derivative = df_derivative.dropna()

fig = go.Figure()

for axis in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x = df_derivative.index, y = df_derivative[axis], name = axis))

fig.show()

In [ ]:
df.index = pd.to_datetime(df['time'], unit = 'ns')
fig = go.Figure()

for axis in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x = df_derivative.index, y = df_derivative[axis], name = axis))

fig.show()

In [ ]:
# path_E4 = "./data/output/"
import plotly.express as px

participant_filepath_output = "./data/output/"
for participant_filepath in glob.glob(participant_filepath_output + '*/', recursive=True):
    light = pd.read_csv('{}Light_annotated.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    # df = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')
    light.index = pd.to_datetime(light['time'], unit = 'ns')
    light = light.dropna(subset=['text'])

    # Create a figure with 3 subplots using the subplots function
    fig = sp.make_subplots(rows=3, cols=1)

   # Use plotly to create the line plot
    fig = px.line(light, x='time', y='lux', color='text', labels={'light_sensor':'Light Sensor Data'})
    fig.update_layout(xaxis=dict(tickvals=[], showticklabels=False))

    # Show the plot
    fig.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
participant_filepath_output = "./data/output/"
# Create an empty dictionary to store the count of text column for each participant
participant_counts = {}

# Generate a list of ten colors using a Seaborn color palette
colors = sns.color_palette("husl", 10)

# Increase the saturation of the colors to make them more vibrant
colors = [sns.desaturate(c, 0.8) for c in colors]

# Loop through all participant files
for participant_filepath in glob.glob(participant_filepath_output + '*/', recursive=True):
    # read in the accelerometer data
    accelerometer = pd.read_csv('{}Accelerometer_annotated.csv'.format(participant_filepath))

    # extract the participant ID from the file path
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]

    # group the data by the text column and count the number of occurrences of each category
    text_counts = accelerometer.groupby('Label').size()

    # store the counts in the participant_counts dictionary
    participant_counts[participant] = text_counts

# Convert the participant_counts dictionary into a Pandas DataFrame
df = pd.DataFrame.from_dict(participant_counts, orient='index')

# Plot a stacked bar chart
ax = df.plot(kind='bar', stacked=True, figsize=(10,5), color=colors)
ax.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
ax.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0), title='Activity Type')
# plt.title('Count of activities')
plt.xlabel('Participant ID')
plt.ylabel('Count')
plt.savefig(path_img + 'distribution of tasks.pdf',bbox_inches='tight')
plt.show()

In [ ]:
for participant_filepath in glob.glob(filepath_sensors + '2023-02-15_15-33-11/', recursive=True):
    location = pd.read_csv('{}Location.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    location.index = pd.to_datetime(location['time'], unit = 'ns')

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=location.index, y=location['speed'], mode='markers',
              error_y={'type': 'data', 'array': location['speedAccuracy']}))
    fig.update_layout(legend_title_text = "Location")
    fig.update_xaxes(title_text="Timestamp_" + participant)
    fig.update_yaxes(title_text="Location")
    # matplotlib.rcParams['figure.figsize'] = [40, 10]
    fig.show()

In [ ]:
for participant_filepath in glob.glob(filepath_sensors + '2023-02-15_15-33-11/', recursive=True):
    df = pd.read_csv('{}Location.csv'.format(participant_filepath))
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    coords = [(row.latitude, row.longitude) for _, row in df.iterrows()]

    my_map = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=16)
    folium.PolyLine(coords, color="blue", weight=5.0).add_to(my_map)
    display(my_map)